# Лабораторная работа №2
## Выполнил студент группы БФИ2001 Стеклов М. А.

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
3. [Задание 3](#Задание-№3)

### Задание №1
Реализовать методы поиска в соответствии с заданием. Организовать генерацию начального набора случайных данных. Для всех вариантов добавить реализацию добавления, поиска и удаления элементов. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

In [84]:
from random import randint
def get_rand_list(n: int, min_value:int, max_value:int) -> list[int]:
    return [randint(min_value, max_value) for _ in range(n)]

In [85]:
x = get_rand_list(10, -100, 100)
sorted_x = sorted(x)

In [99]:
from abc import ABC, abstractmethod

class SearchClass(ABC):
    def __init__(self, x: list[int]=[]): # x должен быть осортированным списком
        self._x: list[int] = x.copy()
        super().__init__()
            
    @abstractmethod
    def _find_insertion_index(self, el: int) -> int:
        pass
        
    def find(self, el: int) -> int:
        i = self._find_insertion_index(el)
        assert el == self._x[i], "Element not found"
        return i
    
    def insert(self, el: int) -> int:
        index = self._find_insertion_index(el)
        self._x.insert(index, el)
        return index
    
    def remove(self, el: int) -> int:
        i = self.find(el)
        self._x.pop(i)
        return i
    
    def __str__(self) -> str:
        return str(self._x)
        

In [100]:
#Бинарный поиск
class BinarySearch(SearchClass):
    def _find_insertion_index(self, el: int) -> int:
        n = len(self._x)
        l, r = 0, n - 1
        m = (l + r) // 2
        while r - l > 1:
            if el <= self._x[m]:
                r = m
            else:
                l = m
            m = (l + r) // 2
        return r

In [101]:
#Бинарное дерево
from __future__ import annotations

class BinarySearchTree:
    def __init__(self, key: int=None, value: int=None, parent=None):
        self.left: BinarySearchTree = None
        self.right: BinarySearchTree = None
        self.parent: BinarySearchTree = parent
        if key is None or value is None:
            self.key = None
            self.value = None
        else:
            self.key = key
            self.value = value
            
    # Поиск
    def find(self, key: int) -> BinarySearchTree:
        assert key is not None, "Key can't be None"
        assert self.key is not None, f"Element with key {key} not found"
        if key == self.key:
            return self
        return self.left.find(key) if key < self.key else self.right.find(key)
    
    # Вставка
    def insert(self, key: int, value: int):
        assert key is not None, "Key can't be None"
        if self.left is None:
            self.left = BinarySearchTree(parent=self)
        if self.right is None:
            self.right = BinarySearchTree(parent=self)
        if self.key is None or self.key == key:
            self.key = key 
            self.value = value
            return
        if key < self.key:
            self.left.insert(key, value)
        else:
            self.right.insert(key, value)
    
    def find_min(self):
        if self.left.key == None:
            return self
        return self.left.find_min()
    
    # Удаление
    def remove(self, key: int):
        assert key is not None, "Key can't be None"
        assert self.key is not None, f"Element with key {key} not found"
        if key < self.key:
            self.left.remove(key)
        elif key > self. key:
            self.right.remove(key)
        else:
            if self.left.key is None:
                if self.right.key is None:
                    self.key = None
                    self.value = None
                    self.left = None
                    self.right = None
                else:
                    if self.key < self.parent.key:
                        self.parent.left = self.right
                    else:
                        self.parent.right = self.right
            else:
                if self.right.key is None:
                    if self.key < self.parent.key:
                        self.parent.left = self.left
                    else:
                        self.parent.right = self.left
                else:
                    next_node = self.right.find_min()
                    self.key = next_node.key
                    self.value = next_node.value
                    next_node.remove(next_node.key)
    
    def __str__(self) -> str:
        if self.key is None:
            return ''
        str_value = (('' if self.left.key is None else '_')+
                     str(self.key) +
                     ('' if self.right.key is None else '_'))
        if self.left.key is None and self.right.key is None:
            return str_value
        split_left = str(self.left).split('\n')
        len_left = len(split_left)
        split_right = str(self.right).split('\n')
        len_right = len(split_right)
        if len_right > len_left:
            split_left += [' ' * len(split_left[0])
                               for _ in range(len_right - len_left)]
        else:
            split_right += [' ' * len(split_right[0])
                                for _ in range(len_left - len_right)]
        value_offset = ' ' * len(str_value)
        str_children = '\n'.join(map(lambda x: x[0] + value_offset + x[1],
                                     zip(split_left, split_right)))
        left_offset = ' ' * len(split_left[0])
        for i in range(len(split_left[0]) - 1, -1, -1):
            if split_left[0][i] not in (' ', '_'):
                left_offset = ' ' * i + '_' * (len(split_left[0]) - i)
                break
        right_offset = ' ' * len(split_right[0])
        for i in range(len(split_right[0])):
            if split_right[0][i] not in (' ', '_'):
                right_offset = '_' * (i + 1) + ' ' * (len(split_right[0]) - i - 1)
                break
        return left_offset + str_value + right_offset + '\n' + str_children
        

In [102]:
tree = BinarySearchTree()
for el in x:
    tree.insert(el, el)
print(tree, end='\n\n')
tree.insert(15, 15)
print(tree, end='\n\n')
print(tree.find(tree.left.key), end='\n\n')
tree.remove(15)
print(tree, end='\n\n')
    


-89_________________          
      _____________33__       
    -83______         41__    
          __-23__        64__ 
        -40     16          89

-89____________________          
      ________________33__       
    -83______            41__    
          __-23_____        64__ 
        -40      __16          89
                15               



AssertionError: Key can't be None

In [103]:
#Метод Фибоначчи

In [110]:
#Интерполяционный
class InterpolationSearch(SearchClass):
    def _find_insertion_index(self, el: int) -> int:
        x = self._x
        n = len(x)
        l, r = 0, n - 1
        while x[l] < el < x[r]:
            m = l + (el-x[l]) * (r-l) // (x[r]-x[l])
            if x[m] < el:
                l = m + 1
            elif x[m] > el:
                r = m - 1
            else:
                return m
        if x[l] == el:
            return l
        elif x[r] == el: 
            return r
        else:
            return r + 1 if x[r] - el < el - x[l] else l

In [109]:
a = InterpolationSearch(sorted_x)
print(sorted_x)
a.insert(1)
a.remove(1)
print(a)

[-89, -83, -40, -23, -23, 16, 33, 41, 64, 89]
[-89, -83, -40, -23, -23, 16, 33, 41, 64, 89]


### Задание №2
Написать соответствующие решения для заданных методов.

In [1]:
from math import sqrt
#Простое рехэширование
class HashTable:
    def __init__():
        self.n = 3
        self.size = 2**n
        self.const = int(self.size*(sqrt(5) - 1)/2)
        self.values = [None] * self.size
        self.values_count = 0
    
    def insert(self, value):
        i = self.get_hash(value)
        if values[i] is None:
            values[i] = value
        else values[i] != value:
            
    
    def find(self):
        pass
    
    def remove(self):
        pass
    
    def _calc_size_const(self):
        self.size = 2**n
        self.const = int(self.size*(sqrt(5) - 1)/2)
    
    def _get_hash(self):
        

SyntaxError: invalid syntax (3896094921.py, line 3)

In [ ]:
#Рехэширование с помощью псевдослучайных чисел


In [ ]:
#Метод цепочек

### Задание №3
Расставить на стандартной 64-клеточной шахматной доске 8 ферзей так, чтобы ни один из них не находился под боем другого». Подразумевается, что ферзь бьёт все клетки, расположенные по вертикалям, горизонталям и обеим диагоналям
Написать программу,  которая находит хотя бы один способ решения задач.


In [111]:
def helper(board: set[tuple[int, int]], depth=0) -> tuple[bool, list[tuple[int, int]]]:
    if depth == 8:
        return (True, [])
    if len(board) == 0:
        return (False, [])
    my_board = board.copy()
    for i, j in board:
        my_board -= {(i, _j) for _j in range(8)}
        my_board -= {(_i, j) for _i in range(8)}
        d = i - j
        i_s, j_s = (0, -d) if d <= 0 else (d, 0)
        my_board -= {(i_s + s, j_s + s) for s in range(min(8 - i_s, 8 - j_s))}
        d = 7 - j - i
        i_s, j_s = (-d, 7) if d <= 0 else (0, 7 - d)
        my_board -= {(i_s + s, j_s - s) for s in range(min(8 - i_s, j_s))}
        if (res := helper(my_board, depth=depth + 1))[0]:
            res[1].append((i, j))
            return res
        my_board = board.copy()
    return (False, [])

from IPython.display import display, Markdown
def task3():
    board = {(i, j) for i in range(8) for j in range(8)}
    board = helper(board)[1]
    out_board = [[(lambda y, x: '▢' if (i + j) % 2 == 0 else '▣')(i, j)
                  for j in range(8)] for i in range(8)]
    for i, j in board:
        out_board[i][j] = '♕' if (i + j) % 2 == 0 else '♛'
    display(Markdown("<br>".join(map(lambda x: f"""<font size="5">{' '.join(x)}</font>""", out_board))))
task3()

<font size="5">▢ ▣ ♕ ▣ ▢ ▣ ▢ ▣</font><br><font size="5">▣ ▢ ▣ ▢ ♛ ▢ ▣ ▢</font><br><font size="5">▢ ♛ ▢ ▣ ▢ ▣ ▢ ▣</font><br><font size="5">▣ ▢ ▣ ▢ ▣ ▢ ▣ ♕</font><br><font size="5">♕ ▣ ▢ ▣ ▢ ▣ ▢ ▣</font><br><font size="5">▣ ▢ ▣ ▢ ▣ ▢ ♛ ▢</font><br><font size="5">▢ ▣ ▢ ♛ ▢ ▣ ▢ ▣</font><br><font size="5">▣ ▢ ▣ ▢ ▣ ♕ ▣ ▢</font>